## 행정안전부 도로명주소 API 활용

In [1]:
import requests, json       # requests: 인터넷으로 자료를 받아서  가지고 올때 사용(크롤링때씀)
import pandas as pd
from urllib.parse import quote  # quote: 인터넷 주소창에 한글 인코딩할때 사용

- API Key 가져오기

In [2]:
with open('keys/국토교통apikey.txt') as file:
    road_key = file.read()

- URL 만들기

In [3]:
# requests get method를 이용하여 데이터를 보내는 경우에는 한글을 반드시 인코딩해주어야 한다.
quote('영등포역')

'%EC%98%81%EB%93%B1%ED%8F%AC%EC%97%AD'

![Alt text](image.png)

![Alt text](image-1.png)

In [5]:
url = 'http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=d4f6868047684ca680cd1ef7e8a15b4a&form_id=996&style_num=1&start_dt=2022&end_dt=2022'

- Open API를 호출하여 결과 받기

In [6]:
result = requests.get(url)
result.status_code      # 결과값이 200이면 제대로 받아온것

200

In [7]:
url

'http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=d4f6868047684ca680cd1ef7e8a15b4a&form_id=996&style_num=1&start_dt=2022&end_dt=2022'

In [8]:
result.text[:200]

'{\n  "result_status": {\n    "status_code": "INFO-000",\n    "message": "정상 처리되었습니다."\n  },\n  "result_data": {\n    "unitName": "대/일,%",\n    "formName": "1.도로종류별 노선별 차종별 평균일교통량과 구성비",\n    "cert_id": "d4f68'

- JSON data를 python에서 읽을 수 있도록 변환

In [9]:
res = json.loads(result.text)

- 딕셔너리 데이터에서 필요한 정보 추출하기

In [10]:
res.keys()

dict_keys(['result_status', 'result_data'])

In [12]:
res['result_data'].keys()

dict_keys(['unitName', 'formName', 'cert_id', 'formList'])

In [13]:
type(res['result_data']['formList'])

list

In [14]:
res['result_data']['formList'][0].keys()

dict_keys(['노선번호', '버스비율', 'date', '도로등급', '계', '버스', '승용차', '화물차', '화물차비율', '노선명', '승용차비율'])

In [18]:
# 원하는 결과: 교통량
res['result_data']['formList'][0]['계'],res['result_data']['formList'][0]['버스'],res['result_data']['formList'][0]['승용차'],res['result_data']['formList'][0]['화물차'],res['result_data']['formList'][0]['노선명']

(96492, 3215, 65944, 27333, '경부선')

In [19]:
res

{'result_status': {'status_code': 'INFO-000', 'message': '정상 처리되었습니다.'},
 'result_data': {'unitName': '대/일,%',
  'formName': '1.도로종류별 노선별 차종별 평균일교통량과 구성비',
  'cert_id': 'd4f6868047684ca680cd1ef7e8a15b4a',
  'formList': [{'노선번호': '1',
    '버스비율': 3.3,
    'date': '2022',
    '도로등급': '고속국도',
    '계': 96492,
    '버스': 3215,
    '승용차': 65944,
    '화물차': 27333,
    '화물차비율': 28.3,
    '노선명': '경부선',
    '승용차비율': 68.3},
   {'노선번호': '10',
    '버스비율': 1.3,
    'date': '2022',
    '도로등급': '고속국도',
    '계': 44048,
    '버스': 594,
    '승용차': 28387,
    '화물차': 15067,
    '화물차비율': 34.2,
    '노선명': '남해선',
    '승용차비율': 64.4},
   {'노선번호': '12',
    '버스비율': 1.7,
    'date': '2022',
    '도로등급': '고속국도',
    '계': 22322,
    '버스': 387,
    '승용차': 15378,
    '화물차': 6557,
    '화물차비율': 29.4,
    '노선명': '광주대구선',
    '승용차비율': 68.9},
   {'노선번호': '14',
    '버스비율': 0.7,
    'date': '2022',
    '도로등급': '고속국도',
    '계': 11564,
    '버스': 86,
    '승용차': 7747,
    '화물차': 3731,
    '화물차비율': 32.3,
    '노선명': '함양울산선',
    '승용

- 영등포역, 신도림역, 당산역, 여의도역, 영등포구청역

['영등포역', ' 신도림역', ' 당산역', ' 여의도역', ' 영등포구청역']

In [16]:
road_addr_list = []
for station in stations:
    params2 = f'keyword={quote(station)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
       res = json.loads(result.text)
       road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [17]:
df = pd.DataFrame({
    '이름': stations, '주소':road_addr_list
})
df

,이름,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)


In [18]:
df.to_csv('data/영등포소재 지하철역.csv', index=False)      # index=False이걸해야 언네임드컬럼안뜸!! 기억!!
pd.read_csv('data/영등포소재 지하철역.csv')

,이름,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)
